Cosas a hacer:
- Traerme las cosas del EDA.ipynb
- Añadir columnas de feature engineering, como menor de edad, español, estancia_larga (si la estancia en días es mayor a 7), dummy encoding de la categoría, 
- Dividir la tabla en subtablas normalizadas
- Meter tabla de, a cada diagnóstico de los principales, asociarles tratamientos, los tratamientos son datos nuevo. También con categorías.

In [ ]:
# Combinar múltiples factores de complejidad
def calcular_complejidad(row):
    score = 0
    score += row['num_diagnosticos_secundarios'] * 0.3
    score += row['num_procedimientos'] * 0.2
    score += (row['nivel_severidad_apr'] - 1) * 0.25
    score += (row['riesgo_mortalidad_apr'] - 1) * 0.25
    return score

# Categorías: Baja, Media, Alta, Muy Alta

Grupos de Riesgo
Bajo Riesgo: Severidad 1, Mortalidad 1, <3 diagnósticos
Riesgo Moderado: Severidad 2, Mortalidad 1-2
Alto Riesgo: Severidad 3-4 o Mortalidad 3-4
Riesgo Crítico: Múltiples factores combinados

2. CATEGORIZACIÓN TEMPORAL Y DEMOGRÁFICA
A) Grupos Etarios Especializados en Salud Mental
Pediátrico (7-17 años)
Joven Adulto (18-30 años) - período crítico para esquizofrenia
Adulto Temprano (31-45 años)
Adulto Medio (46-60 años)
Adulto Mayor (61-75 años)
Geriátrico (76+ años)

B) Estacionalidad Clínica
Temporada Alta (Oct-Ene): Mayor incidencia de trastornos afectivos
Temporada Media (Feb-May, Ago-Sep)
Temporada Baja (Jun-Jul): Menor incidencia

# Basado en estancia, coste y procedimientos
def categorizar_usuario(row):
    if row['estancia_dias'] <= 5 and row['coste_apr'] <= 4500:
        return 'Usuario_Ambulatorio'
    elif row['estancia_dias'] <= 15 and row['num_procedimientos'] <= 2:
        return 'Usuario_Estándar'
    elif row['estancia_dias'] <= 30 and row['coste_apr'] <= 6500:
        return 'Usuario_Intensivo'
    else:
        return 'Usuario_Complejo'

# Agrupación por patología principal + comorbilidades
clusters = {
    'Psicótico_Puro': ['F20', 'F25', 'F29'] + poca comorbilidad,
    'Psicótico_Complejo': ['F20', 'F25'] + alta comorbilidad,
    'Afectivo_Bipolar': ['F31'] + variantes,
    'Afectivo_Depresivo': ['F32', 'F33', 'F34'],
    'Personalidad_Cluster_B': ['F60.3', 'F60.4', 'F60.5'],
    'Adicciones_Comórbidas': Trastornos de sustancias + otros,
    'Neurosis_Ansiedad': ['F40', 'F41', 'F43']
}

B) Niveles de Intervención
Mínima: Sin procedimientos, estancia <7 días
Moderada: 1-3 procedimientos, estancia 7-21 días
Intensiva: >3 procedimientos o estancia >21 días
Crisis: UCI, alta complejidad, riesgo vital

def vulnerabilidad_social(row):
    score = 0
    if row['pais_nacimiento'] not in ['espana', 'desconocido']:
        score += 1
    if 'Z63' in str(row['diagnostico_2']):  # Problemas familiares
        score += 1
    if 'Z59' in str(row['diagnostico_2']):  # Problemas de vivienda
        score += 1
    # etc.

def riesgo_readmision(row):
    # Basado en: diagnóstico, estancia previa, edad, comorbilidades
    if row['categoria'] == 'Esquizofrenia' and row['edad'] < 35:
        return 'Alto'
    elif row['num_diagnosticos_secundarios'] > 6:
        return 'Alto'
    # etc.